In [1]:
import tensorflow as tf
import tensorflow.keras as keras

from PIL import Image

from functools import partial

In [24]:
def decode_image(img):
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.reshape(img, [64, 64, 3])
    
    return img

def _parse_example(example, data_shape):
    tensor = tf.io.parse_single_example(example, data_shape)
    
    datapoint = {
        'anchor': decode_image(tensor['anchor']),
        'pos': decode_image(tensor['pos']),
        'neg': decode_image(tensor['neg'])
    }
    
    return datapoint

In [25]:
dataset = tf.data.TFRecordDataset('KinFaceWITestFolds_jpeg.tfrecords')

In [28]:
data_shape = {
    'anchor': tf.io.FixedLenFeature([], tf.string),
    'pos': tf.io.FixedLenFeature([], tf.string),
    'neg': tf.io.FixedLenFeature([], tf.string)
}

parser = partial(_parse_example, data_shape=data_shape)
dataset = dataset.map(parser).shuffle(2048).batch(128).prefetch(2)

In [32]:
for item in dataset:
    print(item['anchor'].shape)
    break

(128, 64, 64, 3)
